# Decision Tree from Scratch 🌳

## Overview 
In this project, I’m implementing a simple Decision Tree model using Python and NumPy. This model works for both classification and regression tasks by splitting the data at each node based on a feature that best separates the target variable.

### Key Concepts:
- **Decision Tree**: A model that splits data into branches based on feature values to make predictions.
  
- **Entropy**: A measure of impurity or disorder in the data, used to determine the best feature for splitting.
  
- **Information Gain**: The reduction in entropy achieved by partitioning the dataset based on a feature.
  
- **Recursion**: A method used to build the tree by repeating the process of splitting until a stopping condition is met.

---

## Objective 🎯
The goal of this project is to:
1. Implement a custom Decision Tree class for classification tasks.
   
2. Use entropy and information gain to split the dataset at each node.
   
3. Build the decision tree recursively to make predictions for unseen data.

---

## Decision Tree Explanation 🧠

<img src="./figures/decision tree.png" alt="decision tree" width="800" hight= "400"/>


### Decision Tree Structure
A decision tree is a flowchart-like structure where:
- Each internal node represents a "test" on a feature (e.g., "Is feature X ≤ value?").
  
- Each branch represents the outcome of the test.
  
- Each leaf node represents a class label (for classification) or a predicted value (for regression).

### Entropy and Information Gain

**Entropy** is a measure of the disorder or impurity in a set of data. The goal of building a decision tree is to reduce this impurity. The **Information Gain** is the reduction in entropy after a split is made.

#### Entropy Formula:
$$
\text{Entropy}(S) = -p_1 \log_2 p_1 - p_2 \log_2 p_2 - \dots - p_n \log_2 p_n
$$

Where:
- \( p_i \) is the probability of each class in the dataset.

#### Information Gain Formula:
$$
\text{Information Gain}(S, A) = \text{Entropy}(S) - \sum_{v \in \text{Values}(A)} \frac{|S_v|}{|S|} \text{Entropy}(S_v)
$$

Where:
- \( S \) is the original dataset.
- \( A \) is a feature.
- \( S_v \) is the subset of the dataset where feature \( A \) has value \( v \).

---

## Recursive Tree Building 🌳

To build the decision tree, we use recursion:
1. **Split the dataset**: At each step, find the feature that maximizes information gain.

2. **Create child nodes**: Recursively build child nodes by splitting the data until a stopping condition is met.
   
3. **Stopping conditions**: 
   - All data points in a node belong to the same class (for classification) or have the same value (for regression).
   - Maximum tree depth is reached.
   - The number of samples in the node is too small.

---

## Implementation 🛠️

Below is the code for implementing a Decision Tree from scratch. The `DecisionTree` class includes methods to:

1. **Fit the model**: Build the decision tree by recursively splitting the dataset based on the feature that maximizes information gain.
   
2. **Predict**: Make predictions by traversing the decision tree from the root to the leaves.
   
3. **Calculate Entropy and Information Gain**: Compute the entropy and information gain for choosing the best feature for splitting.
---

# Let's code the decision tree from scratch

In [3]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth=None):
        """
        Initialize the Decision Tree Classifier.
        
        Parameters:
        max_depth (int): Maximum depth of the tree. If None, the tree will grow until all leaves are pure or no further splits are possible.
        """
        self.max_depth = max_depth
        self.tree = None  # To store the constructed decision tree

    def fit(self, X, y):
        """
        Train the Decision Tree on the provided dataset.
        
        Parameters:
        X (numpy array): Feature matrix of shape (n_samples, n_features).
        y (numpy array): Target vector of shape (n_samples,).
        """
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        """
        Recursively build the decision tree.

        Parameters:
        X (numpy array): Feature matrix at the current node.
        y (numpy array): Target vector at the current node.
        depth (int): Current depth of the tree.

        Returns:
        dict: A dictionary representing a node in the decision tree.
        """
        # Base case: Stop if all labels are the same or maximum depth is reached
        if len(np.unique(y)) == 1 or (self.max_depth and depth >= self.max_depth):
            return {"class": np.bincount(y).argmax()}  # Majority class at this node

        # Find the best feature and threshold to split the data
        best_split = self._find_best_split(X, y)

        # Recursively build the left and right subtrees
        left_tree = self._build_tree(*best_split["left"], depth + 1)
        right_tree = self._build_tree(*best_split["right"], depth + 1)

        # Return the current node with split information and children
        return {
            "feature": best_split["feature"],  # Index of the feature to split on
            "threshold": best_split["threshold"],  # Value of the feature for the split
            "left": left_tree,  # Left subtree
            "right": right_tree,  # Right subtree
        }

    def _find_best_split(self, X, y):
        """
        Find the best feature and threshold to split the dataset.

        Parameters:
        X (numpy array): Feature matrix.
        y (numpy array): Target vector.

        Returns:
        dict: Information about the best split, including the feature, threshold, and resulting subsets.
        """
        best_gain = -1  # Initialize the best information gain
        best_split = {}  # Store the details of the best split
        n_features = X.shape[1]  # Number of features

        # Loop through each feature
        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])  # Unique values of the feature
            for threshold in thresholds:
                # Create masks for left and right splits
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask
                left_y, right_y = y[left_mask], y[right_mask]

                # Calculate the information gain of this split
                gain = self._information_gain(y, left_y, right_y)
                if gain > best_gain:  # Update best split if this one is better
                    best_gain = gain
                    best_split = {
                        "feature": feature,  # Best feature index
                        "threshold": threshold,  # Best threshold value
                        "left": (X[left_mask], left_y),  # Left split
                        "right": (X[right_mask], right_y),  # Right split
                    }

        return best_split

    def _entropy(self, y):
        """
        Calculate the entropy of a dataset.

        Parameters:
        y (numpy array): Target vector.

        Returns:
        float: Entropy value.
        """
        probs = np.bincount(y) / len(y)  # Class probabilities
        return -np.sum(p * np.log2(p) for p in probs if p > 0)  # Entropy formula

    def _information_gain(self, parent, left, right):
        """
        Calculate the information gain from a split.

        Parameters:
        parent (numpy array): Original dataset target vector.
        left (numpy array): Target vector for the left split.
        right (numpy array): Target vector for the right split.

        Returns:
        float: Information gain value.
        """
        parent_entropy = self._entropy(parent)  # Entropy of the parent node
        n = len(parent)  # Total number of samples
        left_weight = len(left) / n  # Proportion of samples in the left split
        right_weight = len(right) / n  # Proportion of samples in the right split

        # Information gain formula
        return parent_entropy - (left_weight * self._entropy(left) + right_weight * self._entropy(right))

    def predict(self, X):
        """
        Predict the class labels for the input data.

        Parameters:
        X (numpy array): Feature matrix of shape (n_samples, n_features).

        Returns:
        numpy array: Predicted class labels.
        """
        return np.array([self._traverse_tree(x, self.tree) for x in X])

    def _traverse_tree(self, x, tree):
        """
        Traverse the decision tree to make a prediction for a single sample.

        Parameters:
        x (numpy array): Single feature vector.
        tree (dict): The current node of the decision tree.

        Returns:
        int: Predicted class label.
        """
        if "class" in tree:  # If it's a leaf node, return the class
            return tree["class"]

        # Determine which subtree to traverse based on the threshold
        feature_val = x[tree["feature"]]
        if feature_val <= tree["threshold"]:
            return self._traverse_tree(x, tree["left"])  # Go to the left subtree
        else:
            return self._traverse_tree(x, tree["right"])  # Go to the right subtree

In [5]:
# Sample dataset
# X: Feature matrix where each row is a sample and each column is a feature
# y: Target vector with class labels (binary in this case)
X = np.array([[1, 1], [1, 0], [0, 1], [0, 0]])  # Feature matrix
y = np.array([1, 1, 0, 0])  # Target labels

# Initialize the Decision Tree with a maximum depth of 3
dt = DecisionTree(max_depth=3)

# Train the Decision Tree on the dataset
dt.fit(X, y)

# Predict the class labels for the training data
predictions = dt.predict(X)

# Output the predictions
print("Predictions:", predictions) 

Predictions: [1 1 0 0]


/var/folders/qg/mkzrt26d3w3g4nybmsfxhqhm0000gn/T/ipykernel_28360/652398911.py:103: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return -np.sum(p * np.log2(p) for p in probs if p > 0)  # Entropy formula


# When to Use Decision Tree 🌳

Decision trees are versatile machine learning models widely used for both classification and regression tasks. Here are some scenarios where decision trees are particularly effective:

- **Predicting Categorical Outcomes**: Decision trees are ideal for predicting class labels in classification tasks (e.g., spam vs. non-spam, customer churn).  
  
- **Predicting Continuous Outcomes**: Decision trees can also be used in regression tasks to predict continuous values (e.g., predicting house prices, sales forecasts). 
   
- **Handling Non-linear Relationships**: Decision trees can handle non-linear relationships between features and the target variable without requiring complex transformations. 
   
- **Clear Interpretability**: Decision trees are highly interpretable and easy to visualize, making it straightforward to understand how the model makes decisions. 
   
- **Feature Importance**: Decision trees help identify which features are most important for making predictions, providing valuable insights into the data.

# Pros of Decision Trees ✅

- **Easy to Interpret 📖**: The decision tree structure is intuitive and easy to visualize, making it easy to understand how decisions are made.  
  
- **Handles Both Classification and Regression**: Decision trees can be used for both classification and regression tasks, making them versatile for different types of problems.
    
- **No Need for Feature Scaling**: Unlike many other models, decision trees do not require features to be scaled or normalized.  
  
- **Works Well with Non-linear Data**: Decision trees can capture non-linear relationships between features and the target variable, which makes them more flexible than linear models.  
  
- **Handles Missing Data 🔍**: Decision trees can handle missing data effectively in some implementations by using surrogate splits or ignoring missing values during splitting.

# Cons of Decision Trees ❌

- **Overfitting 😓**: Decision trees can easily overfit the data, especially if the tree is too deep. Pruning, cross-validation, or ensemble methods like Random Forest can help mitigate this.
    
    <img src="./figures/overfit.png" alt="gradient" width="900" hight= "400"/> 

    
- **Instability ⚠️**: Small changes in the data can result in a completely different tree, leading to instability. Ensemble methods like Random Forests can address this.  
  
- **Bias with Imbalanced Data ⚖️**: Decision trees can be biased towards the majority class in imbalanced datasets. Techniques like class weights, oversampling, or using metrics like F1-score can help. 
   
- **Can be Greedy**: Decision trees use a greedy algorithm to split nodes, which may not always lead to the globally optimal tree structure.  
  
- **Prone to High Variance**: Without proper tuning, decision trees can have high variance, meaning they may perform well on training data but poorly on unseen data.

# Ensemble Methods 🌟

Random Forest and Gradient Boosted Trees are powerful ensemble methods based on decision trees. They address weaknesses like overfitting and instability by combining multiple trees to improve robustness and accuracy.

## Conclusion 🎯

Decision trees are powerful, interpretable models that can be used for both classification and regression tasks. Their ability to handle non-linear data, provide insights into feature importance, and visualize decisions makes them highly effective. However, care must be taken to avoid overfitting and instability, which can be mitigated using techniques like pruning, cross-validation, or ensemble methods like Random Forests and Gradient Boosted Trees.